### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = 'PRE_PROCESSING'

## Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
time.sleep(3)
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Acessando a telas para efetuar os downloads - Renomeando os arquivo adicionando o nome dos usuários - Movendo para a pasta PRE_PROCESSING

In [ ]:
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in list_routes:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_page_content = navegador.page_source
    site_user = BeautifulSoup(user_page_content, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)        
        print(f'Rota PENDENTE: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)

    else:
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)
        print(f'Rota FINALIZADA: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)
time.sleep(3)
navegador.close()

In [ ]:
## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

files_to_rename_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
files_to_rename_gpx

for user, file_to_rename_gpx in zip(users, files_to_rename_gpx):
    if os.listdir(DOWNLOADS):
        old_path = os.path.join(DOWNLOADS, file_to_rename_gpx)        
        new_name = f"{file_to_rename_gpx.replace('.gpx', '')}__{user}.gpx"        
        new_path = os.path.join(DOWNLOADS,new_name)        
        #print(new_path)
        os.rename(old_path, new_path)

In [ ]:
# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
files_to_move_pre_processing = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for file_to_move_pre_processing in files_to_move_pre_processing:
    src_path = os.path.join(DOWNLOADS, file_to_move_pre_processing)
    dest_path = os.path.join(PRE_PROCESSING, file_to_move_pre_processing)
    try:
        os.rename(src_path,dest_path)
        print(f"Arquivo '{file_to_move_pre_processing}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{file_to_move_pre_processing}': {e}.")

## Executando de uma só vez - Entendo que essa execução deve se dar de uma única vez, pois pode haver alguma falha no momento de renomear os arquivos

In [1]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = 'PRE_PROCESSING'

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
time.sleep(3)
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)


## Baixando os arquivos 
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in list_routes:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_page_content = navegador.page_source
    site_user = BeautifulSoup(user_page_content, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)        
        print(f'Rota PENDENTE: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)

    else:
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)
        print(f'Rota FINALIZADA: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)
time.sleep(3)
navegador.close()


## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

files_to_rename_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
files_to_rename_gpx

for user, file_to_rename_gpx in zip(users, files_to_rename_gpx):
    if os.listdir(DOWNLOADS):
        old_path = os.path.join(DOWNLOADS, file_to_rename_gpx)        
        new_name = f"{file_to_rename_gpx.replace('.gpx', '')}__{user}.gpx"        
        new_path = os.path.join(DOWNLOADS,new_name)        
        #print(new_path)
        os.rename(old_path, new_path)


# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
files_to_move_pre_processing = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for file_to_move_pre_processing in files_to_move_pre_processing:
    src_path = os.path.join(DOWNLOADS, file_to_move_pre_processing)
    dest_path = os.path.join(PRE_PROCESSING, file_to_move_pre_processing)
    try:
        os.rename(src_path,dest_path)
        print(f"Arquivo '{file_to_move_pre_processing}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{file_to_move_pre_processing}': {e}.")

PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632234
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632233
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632232
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632231
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632230
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632229
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632228
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632227
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9632226
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632225
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632224
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632223
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632222
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9632221
FINISHED: htt

# VERIFICAR A FORMA DE LER OS ARQUIVOS QUE ESTÃO NA PASTA DE DOWNLOAD - CONVERTER PARA CSV OU PARQUET E APÓS ESSA CONVERSÃO LEVAR PARA O MINIO

## Lendo arquivos do Bucket - Transformando os arquivos .gpx em um dataframe e CSV e iniciando com PySpark

In [2]:
import os

In [7]:
files_to_pre_processing = [arquivo for arquivo in os.listdir(PRE_PROCESSING) if arquivo.endswith(".gpx")]
files_to_pre_processing

['9632227__Routes_from_sunnypilot_2023_08_19-dev__CHEVROLET_BOLT_EUV_2022__.gpx',
 '9632230__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632218__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632216__Routes_from_sunnypilot_0_9_4_1-release__TOYOTA_RAV4_2019__.gpx',
 '9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx',
 '9632228__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632229__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632223__Routes_from_sunnypilot_2023_08_19-dev__HONDA_ACCORD_2018__.gpx',
 '9632221__Routes_from_sunnypilot_0_9_4_1-release__CHRYSLER_PACIFICA_HYBRID_2019__.gpx',
 '9632220__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632217__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632224__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632233__Routes_from_sunnypil

In [8]:
# pip install gpxpy
from io import BytesIO
import gpxpy 
import gpxpy.gpx 
import pandas as pd
import xml.etree.ElementTree as ET

In [10]:
with open('9632219__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_ELANTRA_2021__.gpx', 'r', encoding='utf-8') as gpx_file:    
    gpx = gpxpy.parse(gpx_file)

In [11]:
gpx

GPX(tracks=[GPXTrack(name='2023-08-22T21:52:56.426764', segments=[GPXTrackSegment(points=[...])])])

## Daqui para baixo é para mover para o Minio


In [ ]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
from io import BytesIO
import gpxpy 
import gpxpy.gpx 
import pandas as pd
from xml.dom import minidom


DOWNLOADS = '/home/thiago/Downloads/'
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

In [ ]:
info_rota = []
arquivos_rotas_gpx = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(BRONZE_LAYER) if arquivo_gpx.object_name.endswith(".gpx")]

In [ ]:
for arquivo_rota_gpx in arquivos_rotas_gpx:    
    try:
        blob_rota_gpx = minioclient.get_object(BRONZE_LAYER, arquivo_rota_gpx.object_name)
        rota_gpx = blob_rota_gpx.data
        rota_gpx = rota_gpx.decode("ISO-8859-1")
        try:
            gpx = gpxpy.parse(rota_gpx)
        
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        info_rota.append({
                        'latitude': point.latitude,
                        'longitude': point.longitude,
                        'elevacao' : point.elevation,
                        'time_point' : point.time})
    
        except gpxpy.gpx.GPXXMLSyntaxException as e:
            print(f"Erro de sintaxe XML no arquivo {arquivo_rota_gpx.object_name}: {e}")
    except S3Error as e:
        print(f"Erro ao acessar o objeto: {e}")    

In [ ]:
info_rota

In [ ]:
type(rota_gpx)

In [ ]:
# pip install pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName('tcc-project').getOrCreate()

In [ ]:
type(gpx)
# Para retornar a quantidade de pontos (lat+lon+ele) em um arquivo arquivo '.gpx' podemos usar get
# Documentação: https://pypi.org/project/gpxpy/
gpx.get_track_points_no()

# Para retornar a faixa de altitude, a fim de se obter extremos de maior elevação e menor elevação no trajeto percorrido - os valores apresentados são em metros acima do nível do mar
gpx.get_elevation_extremes()

# Não endendi para que isso serve
gpx.get_uphill_downhill()


# Obtendo o nome do criador da rota
creator = gpx.creator
creator


# Para exibir o conteudo do arquivo .gpx em formato xml
print(gpx.to_xml()[:1000])


# É possivel verificar quantas rotas/trilhas nosso arquivo .gpx possui
len(gpx.tracks)


# Como no nosso exemplo acima só temos 1 rota/trilha efetuada, podemos acessar por meio de indice no python, 
# Nesse caso se passa o valor [0] para ter uma precisão exata da rota em que estamos trabalhando
gpx.tracks[0]


# Agora vamos acessar os segmentos da nossa rota/trilha
gpx.tracks[0].segments[0]



# Agora podemos acessar os pontos de dados individuais dentro do nosso gpx acessando a matriz de pontos
gpx.tracks[0].segments[0].points[:10]




info_rota = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            info_rota.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevacao' : point.elevation,
                'time_point' : point.time
            })






info_rota[:3]


info_rota_df = pd.DataFrame(info_rota)
info_rota_df.head()





info_rota_df.to_csv('info_rota.csv', index=False)






# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

## Os arquivos que serão movidos da pasta Download para o MINIO_DATA_LAKE precisam estar transformados em Parquet e CSV.

In [ ]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [ ]:
## Aqui os arquivos com a extensão .gpx saem da pasta Download para o bucket MinIO
files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".gpx")]
for name_file in files_to_datalake:
    local_path = os.path.join(DOWNLOADS, name_file)    
    if os.path.isfile(local_path):
        try:
            minioclient.fput_object(BRONZE_LAYER, name_file, local_path)
            print(f"Arquivo {name_file} enviado com sucesso para o bucket.")
            os.remove(local_path) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {name_file} -> Erro: {e}")          

## Códigos não mais usados

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for name_files in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, name_files)
#     destiny_path = os.path.join(DATALAKE_STAGE, name_files)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {name_files} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     local_path = os.path.join(DATALAKE_STAGE, item)
#     print(local_path)
# 
#     if os.path.isfile(local_path):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, local_path)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP ## #
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()

In [ ]:
# ## ESTE TRECHO TEM A FINALIDADE DE BAIXAR OS ARQUIVOS ## #
#list_finished_routes
#list_pending_routes

# //*[@id="content"]/div[2]/div/table/tbody/tr[6]/td - finalizado
# //*[@id="content"]/div[2]/div/span = PENDENTE
# //*[@id="content"]/div[2]/div/table/tbody/tr[4]/td - pendente

# ## Trecho descontinuado -- Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
# navegador = webdriver.Chrome(service=servico)
# for list_route in list_routes:
#     sleep(3)
#     url = list_route[0]
#     navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
#     navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
# navegador.close()